In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
order_products_prior_data=pd.read_csv('/content/drive/My Drive/Grocart_Project/order_products_prior.csv')

In [ ]:
order_products_train_data=pd.read_csv('/content/drive/My Drive/Grocart_Project/order_products_train.csv')

In [ ]:
orders_data=pd.read_csv('/content/drive/My Drive/Grocart_Project/orders.csv')

In [ ]:
products=pd.read_csv('/content/drive/My Drive/Grocart_Project/products.csv')

In [ ]:
order_products_total = pd.concat([order_products_prior_data, order_products_train_data]) 

In [ ]:
order_products_total.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [ ]:
order_products_total.shape

(33819106, 4)

In [ ]:
reorders = order_products_total[order_products_total['reordered'] == 1]
reorders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
3,2,45918,4,1
5,2,17794,6,1
6,2,40141,7,1


In [ ]:
reorders.shape

(19955360, 4)

In [ ]:
orders = orders_data[['order_id', 'user_id']]
user_orders = reorders.merge(orders, on='order_id')
user_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id
0,2,33120,1,1,202279
1,2,28985,2,1,202279
2,2,45918,4,1,202279
3,2,17794,6,1,202279
4,2,40141,7,1,202279


In [ ]:
user_orders.shape

(19955360, 5)

In [ ]:
user_orders['product_count'] = (user_orders['product_id'].value_counts().sort_values(ascending=False)>1)
product_count = user_orders[user_orders['product_count'] == True]
product_count.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,product_count
1,2,28985,2,1,202279,True
2,2,45918,4,1,202279,True
3,2,17794,6,1,202279,True
4,2,40141,7,1,202279,True
5,2,1819,8,1,202279,True


In [ ]:
product_count.shape

(42710, 6)

In [ ]:
product_count = product_count.merge(products,on='product_id')
product_count.head()


,order_id,product_id,add_to_cart_order,reordered,user_id,product_count,product_name,aisle_id,department_id
0,2,28985,2,1,202279,True,Michigan Organic Kale,83,4
1,27,28985,16,1,129389,True,Michigan Organic Kale,83,4
2,288,28985,10,1,137357,True,Michigan Organic Kale,83,4
3,582,28985,3,1,193223,True,Michigan Organic Kale,83,4
4,588,28985,3,1,145181,True,Michigan Organic Kale,83,4


In [ ]:
product_count.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
user_id              0
product_count        0
product_name         0
aisle_id             0
department_id        0
dtype: int64

In [ ]:
product_count.shape

(42710, 9)

In [ ]:
user_product_count = product_count.groupby(['user_id', 'product_name']).size().sort_values(ascending=False)
user_product_count.head()

user_id  product_name                                  
100330   Yerba Mate Orange Exuberance Tea                  3
90250    Honey Graham Sticks                               3
6409     Honest T Pomegranate Red Herbal Tea               3
100330   Organic Grapefruit Ginger Sparkling Yerba Mate    3
         Organic Bluephoria Yerba Mate                     3
dtype: int64

In [ ]:
user_product_count.shape

(42110,)

In [ ]:
user_product_count_col = product_count.groupby(['user_id', 'product_name']).size().sort_values(ascending=False).unstack().fillna(0)
user_product_count_col.head()

product_name,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Greek Strained Yogurt,0% Greek Yogurt Black Cherry on the Bottom,"0% Greek, Blueberry on the Bottom Yogurt",0% Milkfat Greek Yogurt Honey,1 % Lowfat Milk,1 Apple + 1 Mango Fruit Bar,1 Apple + 1 Pear Fruit Bar,1 Liter,1 Ply Paper Towels,1% Low Fat Chocolate Milk,1% Low Fat Milk,1% Lowfat Cottage Cheese,1% Lowfat Milk,1% Lowfat Organic Milk,1% Milk,1% Milkfat Low Fat Buttermilk,1% Milkfat Low Fat Vitamin A & D Milk,1% Milkfat Lowfat Cottage Cheese,1/3 Less Fat Cream Cheese,10 oz Paper Bowls,100 Calorie Per Bag Popcorn,100 Calorie Healthy Pop Butter 1.2 Oz Mini Bags Microwave Pop Corn,100 Calorie Pretzels Packs,100% Apple Juice,100% Apple Juice Original,100% Baby Kale,100% Cacao Unsweetened Chocolate Baking Bar,100% Carrot Juice,100% Colombian Ground Medium Dark Coffee,100% Cranberry Juice,100% Florida Orange Juice,100% Grated Parmesan Cheese,100% Guava Juice,100% Juice,100% Juice Apple Juice,"100% Juice Apple Juice, No Sugar Added",100% Juice Medley Of Fruits,...,Zero Calorie Cola Soda,Zero Calorie Cream Soda,Zero Calorie Creamy Root Beer Stevia Soda,Zero Calorie Grape Soda,Zero Calorie Lemon Lime Soda,Zero Calorie Lemon Lime Sports Drink,Zero Calorie Lemon Lime Twist Soda,Zero Calorie Orange Soda,Zero Calorie Sweetener,Zero Calories Berry Nutrient Enhanced Water,Zero Coke Soda Mini Bottles,Zero Cola,Zero Go-Go Mixed Berry Vitamin Water,Zero Ion4 Grape Sports Drink,Zero Non-Chlorine Bleach,Zero Rise Orange,Zero Root Beer,Zero Soda,Zero Strawberry Lemon,Zero Ultra Energy Drink,Zero XXX Acai Blueberry Pomegranate,Zero calorie Sport Drink,Zesty Dill Spears,Zesty Lemon Hommus,Zesty Pizza Sprouted Flax Snax,Zinfandel,"Zinfandel, California, 2010",Zingers Cakes,Zucchini Banana & Amaranth Organic Baby Food,Zucchini Gingerbread Carrot Smart Cookies,"Zucchini, Spinach & Banana Blend Veggies On-The-Go Stage 2 (6 Months and Up)",from Concentrate Mango Nectar,in Gravy with Carrots Peas & Corn Mashed Potatoes & Meatloaf Nuggets,of Hanover 100 Calorie Pretzels Mini,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Crispy Almonds Cereal,with Olive Oil Mayonnaise Dressing,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
user_product_count_col.shape

(6869, 9314)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine = pd.DataFrame(cosine_similarity(user_product_count_col),index=user_product_count_col.index, columns=user_product_count_col.index)
cosine.head()


user_id,27,66,90,150,155,206,208,214,222,382,451,503,508,583,652,678,756,787,818,824,854,892,917,971,1059,1072,1082,1177,1202,1264,1344,1378,1381,1423,1440,1487,1498,1503,1519,1548,...,204802,204853,204854,204863,204912,204967,204999,205060,205147,205173,205177,205228,205267,205369,205388,205442,205471,205543,205557,205650,205684,205700,205709,205713,205729,205761,205784,205787,205794,205888,205908,205943,205970,205990,206043,206082,206105,206158,206162,206206
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.612372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.176777,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from pandas import DataFrame
def Recommender_System(user_id):
    p = product_count.groupby(['product_name','user_id']).size().sort_values(ascending=False).unstack().fillna(0)
    recommendations = pd.Series(np.dot(p.values,cosine[user_id]), index=p.index)
    print(p.values.shape)
    df = DataFrame (recommendations,columns=['values'])   
    res = df.sort_values(by=['values'],ascending=False)
    last_res = res.drop('values',axis=1)
    return last_res
last_out = Recommender_System(27)
last_out.head()

(9314, 6869)


""
product_name
Total 2% with Strawberry Lowfat Greek Strained Yogurt
Total 0% Nonfat Greek Yogurt
Total 2% Lowfat Greek Strained Yogurt with Peach
Total 0% Raspberry Yogurt
Banana


In [ ]:
products_original = product_count[product_count.user_id == 27]
products_original.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,product_count,product_name,aisle_id,department_id
236,7099,33754,8,1,27,True,Total 2% with Strawberry Lowfat Greek Strained...,120,16
11971,7099,22124,10,1,27,True,Total 0% Raspberry Yogurt,120,16
19788,7099,46676,9,1,27,True,Total 0% Nonfat Greek Yogurt,120,16
20976,7099,33787,7,1,27,True,Total 2% Lowfat Greek Strained Yogurt with Peach,120,16


In [ ]:
product_org = products_original['product_name']
product_org

236      Total 2% with Strawberry Lowfat Greek Strained...
11971                            Total 0% Raspberry Yogurt
19788                         Total 0% Nonfat Greek Yogurt
20976     Total 2% Lowfat Greek Strained Yogurt with Peach
Name: product_name, dtype: object

In [ ]:
product_org.shape

(4,)

In [ ]:
last_out = last_out.head()
last_out

""
product_name
Total 2% with Strawberry Lowfat Greek Strained Yogurt
Total 0% Nonfat Greek Yogurt
Total 2% Lowfat Greek Strained Yogurt with Peach
Total 0% Raspberry Yogurt
Banana


In [ ]:
last_out.shape

(5, 0)

In [ ]:
from flask import Flask, request, jsonify, render_template


In [ ]:
!pip install flask-ngrok

In [ ]:
%mkdir templates -p

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
import pickle 
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/")
def home():
  return render_template('python1.html')
@app.route("/predict",methods=['POST'])
def show():
  id=0
  if request.method == 'POST':
    id1=request.form['customer_id']
    id = int(id1)
    p = product_count.groupby(['product_name','user_id']).size().sort_values(ascending=False).unstack().fillna(0)
    recommendations = pd.Series(np.dot(p.values,cosine[id]), index=p.index)
    df = DataFrame (recommendations,columns=['values'])
    res =  df.sort_values(by=['values'],ascending=False).head()
    last_res = res.drop('values',axis = 1)
  return render_template('result.html',data=last_res.to_html())      
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5c11ac1fdabc.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Jul/2020 09:57:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2020 09:57:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Jul/2020 09:57:16] "POST /predict HTTP/1.1" 200 -
